In [1]:
import numpy as np
import timeit
import os
import ctypes
import numba
import math


In [2]:
np.random.seed(0)
x = np.random.randint(0, 2147483646, 20, dtype=np.int32)
x

array([ 209652396,  398764591,  924231285, 1478610112,  441365315,
       1537364731,  192771779, 1491434855, 1819583497,  530702035,
        626610453, 1650906866, 1879422756, 1277901399, 1682652230,
        243580376, 1991416408, 1171049868, 1646868794, 2051556033])

In [3]:
all_mehods = []

In [4]:
def sumSqrtPurePython(x):
    sum = 0
    for e in x:
        sum += e ** 0.5
    return sum

all_mehods.append({
    'method': 'Pure Python',
    'func': sumSqrtPurePython,
    'times': []
})

# quick test:
sumSqrtPurePython(x)

647829.590943377